In [12]:
%run _bootstrap.py
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import pyarrow, pandas, sys; \
           print('pyarrow', pyarrow.__version__, \
                 '\npandas ', pandas.__version__, \
                 '\npython ', sys.version.split()[0])

pyarrow 16.1.0 
pandas  2.3.0 
python  3.11.8


In [14]:
# Option A – simplest: don't reload any compiled libs
%autoreload 2
%aimport -numpy -pandas -pyarrow -ccxt   # the leading '-' = *exclude*



In [ ]:
from src.data_loader import get_asset_df
from src.resample import resample_ohlcv

# 1. Load the 1-minute data (cached on disk or fetched once from Binance)
df_1m = get_asset_df("BTC/USDT", timeframe="1min")

# 2. Aggregate into 6-hour bars: '360T' is 360 minutes
df_6h = resample_ohlcv(df_1m, rule="15min")

print(df_6h.head())

BadRequest: binance {"code":-1120,"msg":"Invalid interval."}

In [25]:
print(df_1m.index.freq) # should show <30 * Minutes>

None
